<a href="https://colab.research.google.com/github/kowook137/Model_Discriminator_module/blob/youn/FaceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import ResNet18_Weights
import pandas as pd
import os
from PIL import Image
import gc


# 2. Kaggle API 설정
!pip install -q kaggle
from google.colab import files
files.upload()  # kaggle.json 업로드

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 3. 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d mehmoodsheikh/fairface-dataset --force
!unzip -oq fairface-dataset.zip -d fairface


# 1. GPU 메모리 관리 클래스
class GPUMemoryManager:
    @staticmethod
    def find_max_batch_size(dataset, model, device, init_batch=32, max_trials=10):
        model.train()
        model.zero_grad()
        torch.cuda.empty_cache()

        for bs in [init_batch*(2**i) for i in range(max_trials)]:
            try:
                # 테스트 배치 생성
                inputs, labels = [], []
                for _ in range(bs):
                    img, lbl = dataset[_]
                    inputs.append(img)
                    labels.append(lbl)
                inputs = torch.stack(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                # 순전파/역전파 시뮬레이션
                outputs = model(inputs)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                loss.backward()

                # 메모리 정리
                del inputs, labels, outputs, loss
                torch.cuda.empty_cache()
                gc.collect()
                print(f"✅ 배치 크기 {bs}에서 성공")

            except RuntimeError as e:  # OOM 발생 시
                print(f"❌ 배치 크기 {bs}에서 메모리 부족: {str(e)}")
                torch.cuda.empty_cache()
                return bs // 2

        return bs

# 2. 데이터셋 및 모델 정의
class FairFaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.race_map = {
            'White':0, 'Black':1, 'Latino_Hispanic':2,
            'East Asian':3, 'Southeast Asian':4,
            'Indian':5, 'Middle Eastern':6
        }

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx]['file'])
        image = Image.open(img_path).convert('RGB')
        race = self.race_map[self.labels.iloc[idx]['race']]
        if self.transform:
            image = self.transform(image)
        return image, race

# 3. 메인 실행 함수
def main():
    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")

    # 데이터 전처리
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # 데이터셋 초기화
    base_path = '/content/fairface/FairFace'
    trainset = FairFaceDataset(
        csv_file=os.path.join(base_path, 'fairface_label_train.csv'),
        img_dir=base_path,
        transform=transform
    )
    testset = FairFaceDataset(
    csv_file = os.path.join(base_path, 'fairface_label_val.csv'),
    img_dir = base_path,
    transform = transform
    )

    # 모델 초기화
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, 7)
    model = model.to(device)

    # 최적 배치 크기 탐색
    print("\n🔎 최적 배치 크기 탐색 시작...")
    max_batch = GPUMemoryManager.find_max_batch_size(trainset, model, device)
    print(f"\n⚠️ 최종 선택된 배치 크기: {max_batch}")

    # 데이터로더 설정
    trainloader = DataLoader(trainset, batch_size=max_batch, shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=max_batch, shuffle=False, num_workers=2)

    # 학습 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # 학습 루프
    num_epochs = 30
    print("\n🚀 학습 시작...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 순전파
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # 통계 계산
            total_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # 메모리 관리
            del inputs, labels, outputs
            torch.cuda.empty_cache()

        # 에포크 종료 처리
        avg_loss = total_loss / total
        accuracy = 100. * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} | Acc: {accuracy:.2f}%")

    # 모델 저장
    torch.save(model.state_dict(), 'fairface_resnet18.pth')
    print("\n💾 모델 저장 완료: fairface_resnet18.pth")

    # 학습 종료 후 추가
    model.eval()
    test_correct = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs.to(device))
            test_correct += (outputs.argmax(1) == labels.to(device)).sum().item()
    print(f"Test Accuracy: {100*test_correct/len(testset):.2f}%")


if __name__ == "__main__":
    main()





Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/mehmoodsheikh/fairface-dataset
License(s): MIT
 96% 528M/550M [00:01<00:00, 254MB/s]
100% 550M/550M [00:01<00:00, 343MB/s]
사용 중인 디바이스: cuda

🔎 최적 배치 크기 탐색 시작...
✅ 배치 크기 32에서 성공
✅ 배치 크기 64에서 성공
✅ 배치 크기 128에서 성공
✅ 배치 크기 256에서 성공
✅ 배치 크기 512에서 성공
❌ 배치 크기 1024에서 메모리 부족: CUDA out of memory. Tried to allocate 784.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 132.12 MiB is free. Process 11803 has 14.61 GiB memory in use. Of the allocated memory 13.69 GiB is allocated by PyTorch, and 803.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

⚠️ 최종 선택된 배치 크기: 512

🚀 학습 시작...
Epoch [1/30] Loss: 1.4406 | Acc: 44.14%
Epoch [2/30] Loss: 1.1792 | Acc: 54.54%
Epoch [3/30] Loss